In [1]:
import sys
import os
os.chdir("..")
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("data/processed/df_1.csv", parse_dates = ['date'])
df.head(3)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name
0,1,1,2008/2009,1,2008-08-17,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.73,3.4,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.8,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20,Belgium Jupiler League,Belgium,KRC Genk,Beerschot AC
1,1,1,2008/2009,1,2008-08-16,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.95,3.2,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.9,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60,Belgium Jupiler League,Belgium,SV Zulte-Waregem,Sporting Lokeren
2,1,1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.38,3.3,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.5,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.5,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75,Belgium Jupiler League,Belgium,KSV Cercle Brugge,RSC Anderlecht


In [3]:
home = (
        df[['match_api_id','date','home_team_api_id','home_team_goal','away_team_goal']]
        .rename(columns={
            'home_team_api_id':'team_id',
            'home_team_goal':'goals_for',
            'away_team_goal':'goals_against'
        })
    )
away = (
    df[['match_api_id','date','away_team_api_id','away_team_goal','home_team_goal']]
    .rename(columns={
        'away_team_api_id':'team_id',
        'away_team_goal':'goals_for',
        'home_team_goal':'goals_against'
    })
)
home['is_win'] = (home['goals_for'] > home['goals_against']).astype(int)
away['is_win'] = (away['goals_for'] > away['goals_against']).astype(int)

all_matches = pd.concat([home, away]).sort_values('date')
all_matches.reset_index(drop=True, inplace=True)

In [4]:
all_matches.head(5)

,match_api_id,date,team_id,goals_for,goals_against,is_win
0,486263,2008-07-18,10192,1,2,0
1,486263,2008-07-18,9931,2,1,1
2,486264,2008-07-19,9930,3,1,1
3,486264,2008-07-19,10179,1,3,0
4,486266,2008-07-20,7955,1,2,0


In [5]:
weights = np.array([1, 2, 3, 4, 5])

def weighted_rolling(s: pd.Series) -> pd.Series:
        return (
            s.shift(1)
            .rolling(window=len(weights), min_periods=len(weights))
            .apply(lambda arr: np.dot(arr, weights) / weights.sum(), raw=True)
        )
        
# Tính các feature weighted form và goals
all_matches['home_wins_last5'] = all_matches.groupby('team_id')['is_win'].transform(weighted_rolling)
all_matches['home_avg_gs_last5'] = all_matches.groupby('team_id')['goals_for'].transform(weighted_rolling)
all_matches['home_avg_gc_last5'] = all_matches.groupby('team_id')['goals_against'].transform(weighted_rolling)

recent = all_matches[['match_api_id','team_id','home_wins_last5','home_avg_gs_last5','home_avg_gc_last5']]

# Merge trở lại df
df = df.merge(
    recent.rename(columns={'team_id':'home_team_api_id'}),
    on=['match_api_id','home_team_api_id'],
    how='left'
).merge(
    recent.rename(columns={
        'team_id':'away_team_api_id',
        'home_wins_last5':'away_wins_last5',
        'home_avg_gs_last5':'away_avg_gs_last5',
        'home_avg_gc_last5':'away_avg_gc_last5'
    }),
    on=['match_api_id','away_team_api_id'],
    how='left'
)
# Drop 5 trận đầu tiên (không đủ 5 trận lịch sử)
df = df.dropna(subset=[
    'home_wins_last5','home_avg_gs_last5','home_avg_gc_last5',
    'away_wins_last5','away_avg_gs_last5','away_avg_gc_last5'
]).reset_index(drop=True)

In [6]:
df.head(3)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name,home_wins_last5,home_avg_gs_last5,home_avg_gc_last5,away_wins_last5,away_avg_gs_last5,away_avg_gc_last5
0,1,1,2008/2009,10,2008-11-01,492564,8342,8571,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,5.25,9.5,1.25,5.0,10.0,1.3,4.2,8.0,1.25,4.50,10.0,NaN,NaN,NaN,1.35,4.2,7.0,1.27,5.00,10.0,1.30,4.35,8.5,1.25,5.00,10.0,1.29,4.5,9.00,Belgium Jupiler League,Belgium,Club Brugge KV,KV Kortrijk,0.733333,1.533333,1.000000,0.200000,1.0,1.333333
1,1,1,2008/2009,10,2008-10-31,492565,9985,9986,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,5.25,9.5,1.25,5.0,10.0,1.3,4.2,8.0,1.29,4.33,9.0,NaN,NaN,NaN,1.25,4.5,9.5,1.27,5.00,10.0,1.28,4.50,8.5,1.25,5.00,10.0,1.25,5.0,9.00,Belgium Jupiler League,Belgium,Standard de Liège,Sporting Charleroi,0.466667,1.066667,1.200000,0.266667,1.2,1.200000
2,1,1,2008/2009,10,2008-11-02,492566,10000,9991,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,3.20,2.5,2.60,3.2,2.4,2.4,3.2,2.4,2.50,3.20,2.5,NaN,NaN,NaN,2.60,3.1,2.4,2.60,3.25,2.5,2.65,3.30,2.3,2.60,3.25,2.5,2.62,3.2,2.38,Belgium Jupiler League,Belgium,SV Zulte-Waregem,KAA Gent,0.533333,1.933333,1.066667,0.600000,1.8,0.600000


In [7]:
# D) Head-to-head (3 trận gần nhất với Laplace Smoothing)
alpha = 2
p = 0.5

# 1. Tạo key và bảng tạm
df['h2h_key'] = df.apply(lambda x: tuple(sorted([x['home_team_api_id'], x['away_team_api_id']])), axis=1)
h2h = df[['match_api_id','date','h2h_key','home_team_api_id','away_team_api_id','home_team_goal','away_team_goal']].copy()
h2h['home_win'] = (h2h['home_team_goal'] > h2h['away_team_goal']).astype(int)
h2h = h2h.sort_values('date')

# 2. Tính số trận thắng và số trận đối đầu trong 3 lần trước
h2h['h2h_wins_last3'] = h2h.groupby('h2h_key')['home_win'] \
    .transform(lambda s: s.shift(1).fillna(0).rolling(3, min_periods=1).sum())
h2h['h2h_nobs'] = h2h.groupby('h2h_key')['home_win'] \
    .transform(lambda s: s.shift(1).rolling(3, min_periods=1).count())

# 3. Laplace smoothing
h2h['h2h_win_rate_last3'] = (h2h['h2h_wins_last3'] + alpha * p) / (h2h['h2h_nobs'] + alpha)

# 4. Merge vào df
df = df.merge(
    h2h[['match_api_id','h2h_wins_last3','h2h_nobs','h2h_win_rate_last3']],
    on='match_api_id', how='left'
)

In [8]:
df.head(3)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name,home_wins_last5,home_avg_gs_last5,home_avg_gc_last5,away_wins_last5,away_avg_gs_last5,away_avg_gc_last5,h2h_key,h2h_wins_last3,h2h_nobs,h2h_win_rate_last3
0,1,1,2008/2009,10,2008-11-01,492564,8342,8571,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,5.25,9.5,1.25,5.0,10.0,1.3,4.2,8.0,1.25,4.50,10.0,NaN,NaN,NaN,1.35,4.2,7.0,1.27,5.00,10.0,1.30,4.35,8.5,1.25,5.00,10.0,1.29,4.5,9.00,Belgium Jupiler League,Belgium,Club Brugge KV,KV Kortrijk,0.733333,1.533333,1.000000,0.200000,1.0,1.333333,"(8342, 8571)",0.0,0.0,0.5
1,1,1,2008/2009,10,2008-10-31,492565,9985,9986,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,5.25,9.5,1.25,5.0,10.0,1.3,4.2,8.0,1.29,4.33,9.0,NaN,NaN,NaN,1.25,4.5,9.5,1.27,5.00,10.0,1.28,4.50,8.5,1.25,5.00,10.0,1.25,5.0,9.00,Belgium Jupiler League,Belgium,Standard de Liège,Sporting Charleroi,0.466667,1.066667,1.200000,0.266667,1.2,1.200000,"(9985, 9986)",0.0,0.0,0.5
2,1,1,2008/2009,10,2008-11-02,492566,10000,9991,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,3.20,2.5,2.60,3.2,2.4,2.4,3.2,2.4,2.50,3.20,2.5,NaN,NaN,NaN,2.60,3.1,2.4,2.60,3.25,2.5,2.65,3.30,2.3,2.60,3.25,2.5,2.62,3.2,2.38,Belgium Jupiler League,Belgium,SV Zulte-Waregem,KAA Gent,0.533333,1.933333,1.066667,0.600000,1.8,0.600000,"(9991, 10000)",0.0,0.0,0.5


In [9]:
team_attr = pd.read_csv("data/raw/Team_Attributes.csv", parse_dates=['date'])

attrs = [
        'buildUpPlaySpeed','buildUpPlayPassing',
        'chanceCreationPassing','chanceCreationShooting',
        'defencePressure','defenceAggression','defenceTeamWidth'
    ]

# 1) Hàm lấy prev_season từ "YYYY/YYYY+1"
def prev_season_label(season_str: str) -> str:
    y0, y1 = map(int, season_str.split('/'))
    return f"{y0-1}/{y1-1}"

df['prev_season'] = df['season'].apply(prev_season_label)

# 2) Tạo season label đúng kiểu châu Âu cho team_attr
def make_season_label(d: pd.Timestamp) -> str:
    start = d.year if d.month >= 7 else d.year-1
    return f"{start}/{start+1}"

team_attr['season'] = team_attr['date'].apply(make_season_label)

# 3) Lấy bản ghi cuối mỗi (team_api_id, season)
team_attr_last = (
team_attr
.sort_values(['team_api_id', 'season', 'date'])
.drop_duplicates(subset=['team_api_id','season'], keep='last')
.loc[:, ['team_api_id','season'] + attrs]
)

# 4) Merge home / away
for side in ('home','away'):
    df = df.merge(
        team_attr_last.rename(columns={
            'team_api_id':           f'{side}_team_api_id',
            'season':                'prev_season',
            **{col: f'{side}_{col}' for col in attrs}
        }),
        on=[f'{side}_team_api_id','prev_season'],
        how='left'
    )

In [10]:
df.tail(3)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name,home_wins_last5,home_avg_gs_last5,home_avg_gc_last5,away_wins_last5,away_avg_gs_last5,away_avg_gc_last5,h2h_key,h2h_wins_last3,h2h_nobs,h2h_win_rate_last3,prev_season,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,away_chanceCreationPassing,away_chanceCreationShooting,away_defencePressure,away_defenceAggression,away_defenceTeamWidth
24958,24558,24558,2015/2016,9,2015-09-23,1992093,9956,10179,2,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,157856.0,274779.0,177689.0,294256.0,42258.0,39979.0,173936.0,147959.0,451983.0,80564.0,289472.0,10637.0,67349.0,202663.0,32597.0,114794.0,188114.0,25840.0,482200.0,95230.0,451335.0,275122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Switzerland,Grasshopper Club Zürich,FC Sion,0.533333,3.000000,2.000000,0.733333,2.200000,0.8,"(9956, 10179)",0.0,3.0,0.2,2014/2015,50.0,54.0,52.0,46.0,36.0,43.0,47.0,61.0,37.0,62.0,53.0,45.0,47.0,54.0
24959,24558,24558,2015/2016,9,2015-09-22,1992094,7896,10243,0,0,1.0,2.0,4.0,6.0,8.0,3.0,7.0,5.0,3.0,5.0,7.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,NaN,8881.0,173534.0,39646.0,282287.0,340790.0,393337.0,8893.0,614454.0,93229.0,178142.0,274776.0,121080.0,197757.0,260964.0,231614.0,113235.0,41116.0,462608.0,42262.0,92252.0,194532.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Switzerland,Lugano,FC Zürich,0.133333,0.400000,2.266667,0.066667,1.933333,3.2,"(7896, 10243)",0.0,0.0,0.5,2014/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,46.0,47.0,54.0,47.0,43.0,56.0
24960,24558,24558,2015/2016,9,2015-09-23,1992095,10192,9931,4,3,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,274787.0,492132.0,108451.0,25815.0,94553.0,384376.0,598355.0,36785.0,45174.0,302079.0,71764.0,156175.0,95216.0,172768.0,22834.0,458806.0,207234.0,25772.0,40274.0,34035.0,41726.0,527103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [11]:
# Set pandas display options to show all columns
pd.set_option('display.max_rows', None)  # Show all rows
null_counts = df.isnull().sum() 
print(null_counts)

country_id                         0
league_id                          0
season                             0
stage                              0
date                               0
match_api_id                       0
home_team_api_id                   0
away_team_api_id                   0
home_team_goal                     0
away_team_goal                     0
home_player_X1                  1620
home_player_X2                  1620
home_player_X3                  1621
home_player_X4                  1621
home_player_X5                  1621
home_player_X6                  1621
home_player_X7                  1621
home_player_X8                  1621
home_player_X9                  1621
home_player_X10                 1621
home_player_X11                 1621
away_player_X1                  1621
away_player_X2                  1621
away_player_X3                  1621
away_player_X4                  1621
away_player_X5                  1621
away_player_X6                  1621
a

In [12]:
df['diff_speed']    = df['home_buildUpPlaySpeed']    - df['away_buildUpPlaySpeed']
df['diff_shooting'] = df['home_chanceCreationShooting'] - df['away_chanceCreationShooting']
df['diff_pressure'] = df['home_defencePressure']     - df['away_defencePressure']

In [13]:
df.tail(3)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name,home_wins_last5,home_avg_gs_last5,home_avg_gc_last5,away_wins_last5,away_avg_gs_last5,away_avg_gc_last5,h2h_key,h2h_wins_last3,h2h_nobs,h2h_win_rate_last3,prev_season,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,away_chanceCreationPassing,away_chanceCreationShooting,away_defencePressure,away_defenceAggression,away_defenceTeamWidth,diff_speed,diff_shooting,diff_pressure
24958,24558,24558,2015/2016,9,2015-09-23,1992093,9956,10179,2,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,157856.0,274779.0,177689.0,294256.0,42258.0,39979.0,173936.0,147959.0,451983.0,80564.0,289472.0,10637.0,67349.0,202663.0,32597.0,114794.0,188114.0,25840.0,482200.0,95230.0,451335.0,275122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Switzerland,Grasshopper Club Zürich,FC Sion,0.533333,3.000000,2.000000,0.733333,2.200000,0.8,"(9956, 10179)",0.0,3.0,0.2,2014/2015,50.0,54.0,52.0,46.0,36.0,43.0,47.0,61.0,37.0,62.0,53.0,45.0,47.0,54.0,-11.0,-7.0,-9.0
24959,24558,24558,2015/2016,9,2015-09-22,1992094,7896,10243,0,0,1.0,2.0,4.0,6.0,8.0,3.0,7.0,5.0,3.0,5.0,7.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,NaN,8881.0,173534.0,39646.0,282287.0,340790.0,393337.0,8893.0,614454.0,93229.0,178142.0,274776.0,121080.0,197757.0,260964.0,231614.0,113235.0,41116.0,462608.0,42262.0,92252.0,194532.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Switzerland,Lugano,FC Zürich,0.133333,0.400000,2.266667,0.066667,1.933333,3.2,"(7896, 10243)",0.0,0.0,0.5,2014/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,46.0,47.0,54.0,47.0,43.0,56.0,NaN,NaN,NaN
24960,24558,24558,2015/2016,9,2015-09-23,1992095,10192,9931,4,3,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,274787.0,492132.0,108451.0,25815.0,94553.0,384376.0,598355.0,36785.0,45174.0,302079.0,71764.0,156175.0,95216.0,172768.0,22834.0,458806.0,207234.0,25772.0,40274.0,34035.0,41726.0,527103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [14]:
df.shape

(24961, 146)

In [15]:
players = pd.read_csv("data/raw/Player.csv", parse_dates=['birthday'])
recs = []
for side in ['home','away']:
    for i in range(1,12):
        recs.append(
            df[['match_api_id','date',f'{side}_player_{i}']]
            .rename(columns={f'{side}_player_{i}':'player_api_id'})
            .assign(side=side)
        )
long = pd.concat(recs, ignore_index=True)
long = long.merge(
    players[['player_api_id','birthday','height','weight']],
    on='player_api_id', how='left'
)
long['age'] = (long['date'] - long['birthday']).dt.days // 365
agg = (
    long
    .groupby(['match_api_id','side'])
    .agg(
        avg_height=('height','mean'),
        avg_weight=('weight','mean'),
        avg_age=('age','mean')
    )
    .reset_index()
)
pivot = agg.pivot(index='match_api_id', columns='side')
pivot.columns = [f"{side}_{metric}" for metric, side in pivot.columns]
pivot = pivot.reset_index()
df = df.merge(pivot, on='match_api_id', how='left')

In [16]:
long.tail(3)

,match_api_id,date,player_api_id,side,birthday,height,weight,age
549139,1992093,2015-09-23,275122.0,away,1993-04-03,182.88,172.0,22.0
549140,1992094,2015-09-22,194532.0,away,1990-08-30,185.42,181.0,25.0
549141,1992095,2015-09-23,527103.0,away,1997-02-14,185.42,185.0,18.0


In [17]:
df.tail(3)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name,home_wins_last5,home_avg_gs_last5,home_avg_gc_last5,away_wins_last5,away_avg_gs_last5,away_avg_gc_last5,h2h_key,h2h_wins_last3,h2h_nobs,h2h_win_rate_last3,prev_season,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,away_chanceCreationPassing,away_chanceCreationShooting,away_defencePressure,away_defenceAggression,away_defenceTeamWidth,diff_speed,diff_shooting,diff_pressure,away_avg_height,home_avg_height,away_avg_weight,home_avg_weight,away_avg_age,home_avg_age
24958,24558,24558,2015/2016,9,2015-09-23,1992093,9956,10179,2,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,157856.0,274779.0,177689.0,294256.0,42258.0,39979.0,173936.0,147959.0,451983.0,80564.0,289472.0,10637.0,67349.0,202663.0,32597.0,114794.0,188114.0,25840.0,482200.0,95230.0,451335.0,275122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Switzerland,Grasshopper Club Zürich,FC Sion,0.533333,3.000000,2.000000,0.733333,2.200000,0.8,"(9956, 10179)",0.0,3.0,0.2,2014/2015,50.0,54.0,52.0,46.0,36.0,43.0,47.0,61.0,37.0,62.0,53.0,45.0,47.0,54.0,-11.0,-7.0,-9.0,181.494545,182.187273,169.272727,170.636364,26.363636,25.181818
24959,24558,24558,2015/2016,9,2015-09-22,1992094,7896,10243,0,0,1.0,2.0,4.0,6.0,8.0,3.0,7.0,5.0,3.0,5.0,7.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,NaN,8881.0,173534.0,39646.0,282287.0,340790.0,393337.0,8893.0,614454.0,93229.0,178142.0,274776.0,121080.0,197757.0,260964.0,231614.0,113235.0,41116.0,462608.0,42262.0,92252.0,194532.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Switzerland,Lugano,FC Zürich,0.133333,0.400000,2.266667,0.066667,1.933333,3.2,"(7896, 10243)",0.0,0.0,0.5,2014/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,46.0,47.0,54.0,47.0,43.0,56.0,NaN,NaN,NaN,181.263636,178.816000,166.909091,164.700000,26.000000,24.700000
24960,24558,24558,2015/2016,9,2015-09-23,1992095,10192,9931,4,3,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,274787.0,492132.0,108451.0,25815.0,9455

In [18]:
player_attr = pd.read_csv("data/raw/Player_Attributes.csv", parse_dates=['date'])
player_attr.isnull().sum()

id                        0
player_fifa_api_id        0
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    6869
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle     

In [19]:
# Tính match_importance
df['match_importance'] = df['stage'] / df.groupby(['league_id','season'])['stage'].transform('max')

# Chia 3 pha (early / mid / late) theo mùa
df['pct_stage'] = df.groupby(['league_id','season'])['stage'].rank(pct=True)
df['match_phase'] = (
    pd.qcut(
        df['pct_stage'],
        q=[0, 0.33, 0.66, 1.0],
        labels=[1, 2, 3],
        duplicates='drop'
    ).astype(int)
)
df = df.drop(columns=['pct_stage']) 

In [20]:
# Define the team name you want to query
team = "Everton"

# Filter matches where the team plays at home and sort by date
home_matches = df[(df['home_team_name'] == team) | (df['away_team_name'] == team)].sort_values('date')
pd.set_option('display.max_columns', None)  # Show all columns
# Display the results
home_matches.tail(3)  # Show first 10 matches

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name,home_wins_last5,home_avg_gs_last5,home_avg_gc_last5,away_wins_last5,away_avg_gs_last5,away_avg_gc_last5,h2h_key,h2h_wins_last3,h2h_nobs,h2h_win_rate_last3,prev_season,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,away_chanceCreationPassing,away_chanceCreationShooting,away_defencePressure,away_defenceAggression,away_defenceTeamWidth,diff_speed,diff_shooting,diff_pressure,away_avg_height,home_avg_height,away_avg_weight,home_avg_weight,away_avg_age,home_avg_age,match_importance,match_phase
4493,1729,1729,2015/2016,37,2016-05-07,1989073,8197,8668,3,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,37770.0,49571.0,13423.0,23571.0,43061.0,278343.0,103419.0,319300.0,139671.0,20694.0,286119.0,186054.0,193859.0,516273.0,263653.0,24846.0,101192.0,160713.0,30895.0,215384.0,306913.0,181276.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>36</comment><stats...,1.91,4.0,4.0,1.83,3.5,4.25,1.90,3.45,3.8,1.9,3.75,4.20,1.91,3.98,4.07,1.91,3.20,4.33,NaN,NaN,NaN,1.87,3.9,4.2,NaN,NaN,NaN,NaN,NaN,NaN,England Premier League,England,Leicester City,Everton,0.466667,2.133333,0.733333,0.333333,0.933333,1.666667,"(8197, 8668)",0.0,2.0,0.25,2014/2015,63.0,48.0,47.0,69.0,58.0,62.0,54.0,61.0,42.0,39.0,48.0,52.0,58.0,60.0,2.0,21.0,6.0,181.263636,179.878182,166.545455,168.727273,25.000000,28.818182,0.973684,3
4428,1729,1729,2015/2016,30,2016-05-11,1989008,8472,8668,3,0,1.0,2.0,4.0,6.0,8.0,5.0,2.0,4.0,6.0,8.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,8.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,37233.0,432040.0,93457.0,26108.0,180330.0,158879.0,184822.0,25150.0,109330.0,166019.0,30348.0,186054.0,516273.0,263653.0,213878.0,24846.0,101192.0,23782.0,26151.0,215384.0,160713.0,181276.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corne

In [21]:
from src.features.test import *

In [22]:
Matches = pd.read_csv("data/raw/Match.csv", parse_dates=['date'])
players_attr = pd.read_csv("data/raw/Player_Attributes.csv", parse_dates=['date'])
fifa_stats = optimized_get_fifa_stats(Matches, player_attr)

In [23]:
fifa_stats.tail(3)

,match_api_id,away_avg_overall_rating,home_avg_overall_rating,away_avg_potential,home_avg_potential,away_avg_acceleration,home_avg_acceleration,away_avg_sprint_speed,home_avg_sprint_speed,away_avg_finishing,home_avg_finishing,away_avg_crossing,home_avg_crossing,away_avg_heading_accuracy,home_avg_heading_accuracy,away_avg_dribbling,home_avg_dribbling,away_avg_stamina,home_avg_stamina,away_avg_strength,home_avg_strength,away_avg_interceptions,home_avg_interceptions,away_avg_standing_tackle,home_avg_standing_tackle,away_avg_sliding_tackle,home_avg_sliding_tackle,away_avg_long_passing,home_avg_long_passing,away_avg_short_passing,home_avg_short_passing,away_avg_ball_control,home_avg_ball_control,away_avg_long_shots,home_avg_long_shots,away_avg_shot_power,home_avg_shot_power,away_avg_curve,home_avg_curve,away_avg_free_kick_accuracy,home_avg_free_kick_accuracy,away_avg_volleys,home_avg_volleys,away_avg_penalties,home_avg_penalties
25218,2216670.0,63.272727,63.272727,64.818182,67.363636,62.909091,64.181818,63.545455,67.181818,47.454545,44.272727,43.545455,52.181818,55.363636,52.181818,54.090909,56.363636,65.090909,62.727273,63.090909,63.818182,44.363636,49.545455,43.727273,50.363636,41.272727,45.909091,45.454545,51.818182,53.909091,58.363636,58.090909,59.272727,46.909091,46.454545,56.545455,56.909091,49.181818,51.181818,48.636364,51.000000,45.909091,40.545455,50.090909,51.727273
25219,2216671.0,61.545455,69.454545,66.818182,76.272727,63.090909,69.363636,65.363636,68.454545,42.818182,48.181818,45.181818,53.909091,48.727273,55.454545,53.363636,61.272727,63.909091,65.636364,62.363636,62.000000,44.181818,48.909091,47.363636,50.181818,43.636364,50.090909,47.090909,61.181818,53.454545,64.909091,53.181818,64.727273,46.000000,53.000000,55.272727,60.181818,42.636364,54.363636,37.000000,44.727273,42.545455,45.181818,45.545455,45.909091
25220,2216672.0,62.545455,64.900000,66.000000,70.500000,65.090909,72.100000,67.909091,72.100000,35.272727,46.900000,49.090909,45.800000,55.363636,56.300000,54.090909,56.200000,60.272727,70.300000,64.818182,72.800000,49.727273,50.800000,54.363636,52.400000,50.727273,49.700000,49.363636,55.900000,52.636364,59.300000,56.545455,58.400000,36.818182,48.400000,42.545455,59.400000,44.909091,43.200000,40.363636,37.900000,35.454545,38.900000,41.363636,45.100000


In [24]:
df = df.merge(fifa_stats, on='match_api_id', how='left')

In [26]:
df.tail(3)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,country_name,home_team_name,away_team_name,home_wins_last5,home_avg_gs_last5,home_avg_gc_last5,away_wins_last5,away_avg_gs_last5,away_avg_gc_last5,h2h_key,h2h_wins_last3,h2h_nobs,h2h_win_rate_last3,prev_season,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_buildUpPlaySpeed,away_buildUpPlayPassing,away_chanceCreationPassing,away_chanceCreationShooting,away_defencePressure,away_defenceAggression,away_defenceTeamWidth,diff_speed,diff_shooting,diff_pressure,away_avg_height,home_avg_height,away_avg_weight,home_avg_weight,away_avg_age,home_avg_age,match_importance,match_phase,away_avg_overall_rating,home_avg_overall_rating,away_avg_potential,home_avg_potential,away_avg_acceleration,home_avg_acceleration,away_avg_sprint_speed,home_avg_sprint_speed,away_avg_finishing,home_avg_finishing,away_avg_crossing,home_avg_crossing,away_avg_heading_accuracy,home_avg_heading_accuracy,away_avg_dribbling,home_avg_dribbling,away_avg_stamina,home_avg_stamina,away_avg_strength,home_avg_strength,away_avg_interceptions,home_avg_interceptions,away_avg_standing_tackle,home_avg_standing_tackle,away_avg_sliding_tackle,home_avg_sliding_tackle,away_avg_long_passing,home_avg_long_passing,away_avg_short_passing,home_avg_short_passing,away_avg_ball_control,home_avg_ball_control,away_avg_long_shots,home_avg_long_shots,away_avg_shot_power,home_avg_shot_power,away_avg_curve,home_avg_curve,away_avg_free_kick_accuracy,home_avg_free_kick_accuracy,away_avg_volleys,home_avg_volleys,away_avg_penalties,home_avg_penalties
24958,24558,24558,2015/2016,9,2015-09-23,1992093,9956,10179,2,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,157856.0,274779.0,177689.0,294256.0,42258.0,39979.0,173936.0,147959.0,451983.0,80564.0,289472.0,10637.0,67349.0,202663.0,32597.0,114794.0,188114.0,25840.0,482200.0,95230.0,451335.0,275122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Switzerland,Grasshopper Club Zürich,FC Sion,0.533333,3.000000,2.000000,0.733333,2.200000,0.8,"(9956, 10179)",0.0,3.0,0.2,2014/2015,50.0,54.0,52.0,46.0,36.0,43.0,47.0,61.0,37.0,62.0,53.0,45.0,47.0,54.0,-11.0,-7.0,-9.0,181.494545,182.187273,169.272727,170.636364,26.363636,25.181818,0.25,1,67.272727,67.727273,71.727273,73.181818,67.909091,70.090909,70.909091,73.363636,48.636364,49.818182,56.181818,56.000000,51.636364,57.272727,59.818182,61.090909,71.000000,66.545455,71

In [27]:
df.shape

(24961, 198)